# ServiceWeave Code Documentation Explanation
## Comprehensive Guide to Project Files and Cloud Computing Concepts

---

## Introduction

This notebook provides a comprehensive explanation of each code document in the ServiceWeave project. Each section explains:
- What the file does
- Key cloud computing concepts it demonstrates
- How it relates to our cloud course

---

## 1. Frontend Dockerfile

**File:** `frontend/Dockerfile`

### Overview

This is our **Frontend Dockerfile** - it demonstrates **multi-stage builds**, which is a cloud best practice.

### Stage 1 - Builder

We start with Node.js to compile our React application. We copy dependencies, install them, and build the production bundle. This creates optimized JavaScript files.

In [ ]:
# Stage 1: Build React application
FROM node:18-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm install
COPY . .
RUN npm run build

### Stage 2 - Runtime

Here's the key - we use a completely different base image (Nginx) and only copy the built files from stage 1.

In [ ]:
# Stage 2: Lightweight production runtime
FROM nginx:alpine
COPY --from=builder /app/build /usr/share/nginx/html
COPY nginx.conf /etc/nginx/conf.d/default.conf
EXPOSE 8080
CMD ["nginx", "-g", "daemon off;"]

### Cloud Concepts Demonstrated

**Why this matters for cloud:**
- **Smaller image size** - Final image is ~30MB instead of 1GB. This means faster deployments and lower storage costs.
- **Security** - We don't ship Node.js or build tools to production, reducing attack surface.
- **Efficiency** - Faster container startup times, which is critical when scaling horizontally.
- **Non-root port (8080)** - OpenShift requires non-root ports for security.

---

## 2. Backend Dockerfile

**File:** `backend/Dockerfile`

### Overview

Our **Backend Dockerfile** includes a critical cloud feature - **health checks**.

### Basic Containerization

In [ ]:
FROM node:18-alpine
WORKDIR /app
COPY package*.json ./
RUN npm install --production
COPY server.js .
EXPOSE 5000

### Health Check Configuration

The key part - health checks for automated recovery:

In [ ]:
HEALTHCHECK --interval=10s --timeout=5s --retries=3 \
  CMD node -e "require('http').get('http://localhost:5000/health', (r) => {if (r.statusCode !== 200) throw new Error(r.statusCode)})"

CMD ["npm", "start"]

### Cloud Concepts Demonstrated

**Why health checks matter in cloud:**
- **Automated recovery** - If the health check fails 3 times, Kubernetes kills and restarts the container automatically
- **Load balancing** - Unhealthy containers are removed from the service, preventing 503 errors
- **Zero-downtime deployments** - During rolling updates, new pods must pass health checks before old ones are terminated
- **Self-healing** - A core cloud-native principle

---

## 3. Database Dockerfile

**File:** `database/Dockerfile`

### Overview

The **Database Dockerfile** is simple but demonstrates **container initialization**.

In [ ]:
FROM postgres:15-alpine
COPY init.sql /docker-entrypoint-initdb.d/
ENV POSTGRES_USER=postgres
ENV POSTGRES_PASSWORD=postgres
ENV POSTGRES_DB=products_db
EXPOSE 5432

### Cloud Concepts Demonstrated

- **Immutable infrastructure** - The database schema is defined in code (init.sql), not manually configured
- **Reproducibility** - Anyone can spin up an identical database instantly
- **Environment variables** - Configuration is externalized, following **12-factor app principles**
- **Infrastructure as Code** - When this container starts, PostgreSQL automatically runs `init.sql` to create tables and insert seed data

---

## 4. docker-compose.yaml

**File:** `docker-compose.yaml`

### Overview

**docker-compose.yaml** is our local orchestration file. It shows **service composition**.

### Service Definitions

In [ ]:
version: '3.8'
services:
  product-frontend:
    build:
      context: ./frontend
    ports:
      - "80:8080"  # Host:Container mapping
    depends_on:
      - product-backend  # Service dependencies

In [ ]:
  product-backend:
    build:
      context: ./backend
    environment:
      - DB_HOST=product-database  # Service discovery
    depends_on:
      - product-database

In [ ]:
  product-database:
    build:
      context: ./database
    volumes:
      - postgres-data:/var/lib/postgresql/data  # Persistence

volumes:
  postgres-data:
    driver: local  # Named volume for data persistence

### Cloud Concepts Demonstrated

- **Service dependencies** - `depends_on` ensures containers start in the right order
- **Port mapping** - We map host port 80 to container port 8080
- **Networking** - Docker Compose creates an isolated network where services communicate by name
- **Service discovery** - `DB_HOST=product-database` uses service name, not IP address. Docker's internal DNS resolves it.
- **Named volumes** - Local equivalent of Kubernetes PersistentVolumes for data persistence

---

## 5. openshift-deployment.yaml

**File:** `openshift-deployment.yaml`

### Overview

This is our **production deployment manifest** - the most important file for cloud deployment.

### BuildConfig - CI/CD Integration

In [ ]:
apiVersion: build.openshift.io/v1
kind: BuildConfig
metadata:
  name: product-frontend
spec:
  source:
    type: Binary  # Build from local source
  strategy:
    type: Docker  # Use Dockerfile
  output:
    to:
      kind: ImageStreamTag
      name: product-frontend:latest

**Cloud concept:** **CI/CD integration** - OpenShift builds our Docker images from source code automatically. This demonstrates **continuous deployment**.

### DeploymentConfig - Orchestration

In [ ]:
apiVersion: apps.openshift.io/v1
kind: DeploymentConfig
metadata:
  name: product-frontend
spec:
  replicas: 2  # Horizontal scaling
  selector:
    app: product-frontend
  template:
    metadata:
      labels:
        app: product-frontend
    spec:
      containers:
        - name: product-frontend
          image: product-frontend:latest
          ports:
            - containerPort: 8080
          livenessProbe:  # Restart crashed pods
            httpGet:
              path: /
              port: 8080
            initialDelaySeconds: 30
            periodSeconds: 10
          readinessProbe:  # Only send traffic to ready pods
            httpGet:
              path: /
              port: 8080
            initialDelaySeconds: 5
            periodSeconds: 5

**Cloud concepts:**
- **Horizontal scaling** - `replicas: 2` means 2 identical pods for high availability
- **Liveness probes** - Restart crashed pods automatically
- **Readiness probes** - Only send traffic to pods that are ready
- **Declarative configuration** - We describe the desired state; Kubernetes makes it happen

### Service - Stable Endpoint

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: product-frontend
spec:
  type: ClusterIP  # Internal service
  selector:
    app: product-frontend
  ports:
    - port: 80
      targetPort: 8080

**Cloud concept:** **Service abstraction** - The Service provides a stable endpoint. Even if pods die and restart with new IPs, the Service IP never changes.

### Route - External Access

In [ ]:
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: product-frontend
spec:
  to:
    kind: Service
    name: product-frontend
  tls:
    termination: edge  # HTTPS at the edge
    insecureEdgeTerminationPolicy: Redirect  # HTTP → HTTPS

**Cloud concept:** **TLS termination** - OpenShift handles HTTPS at the edge. This demonstrates **separation of concerns** - our app doesn't deal with certificates.

### PersistentVolumeClaim - Data Persistence

In [ ]:
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: product-db-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi

**Cloud concept:** **Stateful workloads** - Even though containers are ephemeral, data persists on external storage (AWS EBS in our case).

---

## 6. Backend server.js

**File:** `backend/server.js`

### Overview

Our **backend code** demonstrates cloud-native application design.

### Database Connection

In [ ]:
const { Client } = require('pg');

const client = new Client({
  host: process.env.DB_HOST || 'product-database',  // Service discovery
  port: 5432,
  database: process.env.DB_NAME || 'products_db',
  user: process.env.DB_USER || 'postgres',
  password: process.env.DB_PASSWORD || 'postgres'
});

**Cloud concepts:**
- **Environment variables** - Configuration is externalized, not hardcoded. This follows **12-factor app methodology**
- **Service discovery** - `DB_HOST='product-database'` uses DNS instead of IPs
- **Default values** - Works locally and in production with different configs

### Health Check Endpoint

In [ ]:
app.get('/health', (req, res) => {
  res.status(200).json({ status: 'healthy' });
});

This is our **health check endpoint** - Kubernetes calls this every 10 seconds to verify the service is alive.

### API Endpoints

In [ ]:
app.get('/api/products', async (req, res) => {
  try {
    const result = await client.query('SELECT * FROM products');
    res.json(result.rows);
  } catch (err) {
    res.status(500).json({ error: err.message });
  }
});

app.get('/api/products/:id', async (req, res) => {
  try {
    const result = await client.query(
      'SELECT * FROM products WHERE id = $1', 
      [req.params.id]
    );
    res.json(result.rows[0]);
  } catch (err) {
    res.status(500).json({ error: err.message });
  }
});

**Cloud concept:** **Stateless design** - Each request is independent. No session data is stored in the container, making horizontal scaling possible.

---

## 7. Frontend App.js

**File:** `frontend/src/App.js`

### Overview

Our **React frontend** shows cloud service communication.

### Service-to-Service Communication

In [ ]:
import React, { useState, useEffect } from 'react';

function App() {
  const [products, setProducts] = useState([]);

  useEffect(() => {
    // Service discovery - using service name
    fetch('http://product-backend:5000/api/products')
      .then(response => response.json())
      .then(data => setProducts(data))
      .catch(error => console.error('Error:', error));
  }, []);

  return (
    <div className="App">
      {products.map(product => (
        <ProductCard key={product.id} product={product} />
      ))}
    </div>
  );
}

**Cloud concepts:**
- **Service-to-service communication** - Frontend calls backend using service name
- **API-driven architecture** - Frontend and backend are decoupled
- **Client-side routing** - React Router handles navigation without server requests
- **Microservices communication** - Each service has a well-defined API contract

---

## 8. Database init.sql

**File:** `database/init.sql`

### Overview

Our **database initialization script** demonstrates Infrastructure as Code.

### Schema Definition

In [ ]:
-- Create products table
CREATE TABLE IF NOT EXISTS products (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  price NUMERIC(10, 2) NOT NULL,
  description TEXT,
  image_url VARCHAR(512),
  category VARCHAR(50),
  stock INTEGER,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

### Seed Data

In [ ]:
-- Insert sample products
INSERT INTO products (name, price, description, image_url, category, stock) VALUES
  ('Laptop Pro 15', 1299.99, 'High-performance laptop', 'https://...', 'Electronics', 45),
  ('Wireless Headphones', 199.99, 'Noise-cancelling', 'https://...', 'Audio', 120),
  ('Smart Watch', 349.99, 'Fitness tracking', 'https://...', 'Wearables', 78),
  ('USB-C Hub', 79.99, '7-in-1 adapter', 'https://...', 'Accessories', 200),
  ('Mechanical Keyboard', 149.99, 'RGB backlit', 'https://...', 'Peripherals', 95),
  ('4K Webcam', 99.99, 'Professional streaming', 'https://...', 'Video', 150),
  ('Phone Mount', 29.99, 'Adjustable stand', 'https://...', 'Accessories', 300),
  ('Portable Speaker', 89.99, 'Waterproof Bluetooth', 'https://...', 'Audio', 180);

**Cloud concepts:**
- **Infrastructure as Code** - Database schema is version-controlled
- **Idempotency** - `CREATE TABLE IF NOT EXISTS` ensures running this multiple times produces the same result
- **Automated provisioning** - No manual database setup needed
- **Declarative infrastructure** - We define what we want, not how to create it

---

## 9. nginx.conf

**File:** `frontend/nginx.conf`

### Overview

Our **Nginx configuration** handles production serving.

### Configuration

In [ ]:
server {
    listen 8080;  # Non-root port for OpenShift
    server_name localhost;
    
    root /usr/share/nginx/html;
    index index.html;
    
    # SPA routing - all routes serve index.html
    location / {
        try_files $uri $uri/ /index.html;
    }
    
    # Cache static assets
    location ~* \.(js|css|png|jpg|jpeg|gif|ico|svg)$ {
        expires 1y;
        add_header Cache-Control "public, immutable";
    }
}

**Cloud concepts:**
- **Non-root port (8080)** - Security requirement for OpenShift
- **SPA routing** - `try_files $uri /index.html` ensures React Router works
- **Static file serving** - Nginx is optimized for production workloads
- **Caching** - Improves performance and reduces bandwidth
- **Production-ready configuration** - Demonstrates difference from development mode

---

## 10. package.json Files

**Files:** `frontend/package.json` and `backend/package.json`

### Overview

Our **package.json files** define dependencies and build scripts.

### Backend package.json

In [ ]:
{
  "name": "product-backend",
  "version": "1.0.0",
  "main": "server.js",
  "scripts": {
    "start": "node server.js",
    "dev": "nodemon server.js"
  },
  "dependencies": {
    "express": "^4.18.2",
    "pg": "^8.11.0",
    "cors": "^2.8.5"
  },
  "devDependencies": {
    "nodemon": "^3.0.1"
  }
}

### Frontend package.json

In [ ]:
{
  "name": "product-frontend",
  "version": "1.0.0",
  "scripts": {
    "start": "react-scripts start",
    "build": "react-scripts build",
    "test": "react-scripts test"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "react-router-dom": "^6.15.0",
    "react-scripts": "5.0.1"
  }
}

**Cloud concepts:**
- **Dependency management** - Ensures consistent environments across development and production
- **Containerization** - `npm install` runs during Docker build to install dependencies
- **Separation of concerns** - Each service has its own dependencies
- **Microservices independence** - Frontend and backend can use different library versions
- **Build automation** - Scripts defined for build, start, and test operations

---

## Conclusion

### Summary of Cloud Computing Concepts

Every file in this project serves a specific cloud computing purpose:

| File Category | Cloud Concepts Demonstrated |
|--------------|-----------------------------|
| **Dockerfiles** | Containerization, multi-stage builds, health checks, security |
| **docker-compose.yaml** | Local orchestration, service composition, networking |
| **openshift-deployment.yaml** | Production orchestration, horizontal scaling, self-healing, declarative infrastructure |
| **Application code** | Cloud-native design, stateless architecture, service discovery, API-driven design |
| **Configuration files** | Infrastructure as Code, externalized configuration, 12-factor methodology |

### Key Takeaways

1. **Containerization** - Each service is packaged with its dependencies for consistency
2. **Orchestration** - Kubernetes/OpenShift manages deployment, scaling, and recovery
3. **Service Discovery** - Services communicate using DNS names, not IPs
4. **Horizontal Scaling** - Stateless services can scale to meet demand
5. **Self-Healing** - Health checks enable automatic recovery
6. **Infrastructure as Code** - Everything is defined in version-controlled files
7. **Security** - TLS termination, non-root ports, minimal images
8. **Persistence** - PersistentVolumes for stateful data

### Complete Cloud-Native Lifecycle

Together, these files demonstrate:
- **Build** - Dockerfiles and BuildConfigs
- **Deploy** - DeploymentConfigs and Services
- **Scale** - Replica sets and load balancing
- **Maintain** - Health checks and automated recovery
- **Monitor** - Logs and health endpoints

This is a complete, production-ready cloud-native application following industry best practices.

---

**End of Documentation**